In [2]:
from get_datasets import get_data
train_dataset, val_dataset, test_dataset, vocab = get_data('SentimentClassifier/')

In [3]:
from torch.utils.data import DataLoader

def sorted_collate(batch):
    x, y = zip(*batch)
    order = np.argsort([len(xi) for xi in x])[::-1]
    x = [torch.LongTensor(x[i]) for i in order]
    y = torch.stack([y[i] for i in order])[:, None]
    return x,y

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, collate_fn=sorted_collate)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False, collate_fn=sorted_collate)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False, collate_fn=sorted_collate)

In [4]:
from torch import jit
import torch
from torch import nn

class SkipGRUCell(jit.ScriptModule):
    def __init__(self, input_size, hidden_size):
        """Simple SkipGRU cell.
        Args:
            input_size (int): size of input space of GRU
            hidden_size (int): size of hidden space of GRU
        """
        super().__init__()
        self.inner_gru = nn.GRUCell(input_size, hidden_size)

    @jit.script_method
    def forward(self, input, hidden, mix):
        """
        Args:
            input (torch.FloatTensor): input vector for the current step
            hidden (torch.FloatTensor): hidden vector for the previous step
            mix (torch.FloatTensor): mixture vector which defines proportion of mix between previous step and current.
        Returns:
            hidden (torch.FloatTensor): hidden vetor for the current step
        """
        # type: (Tensor, Tensor, Tensor) -> Tensor

        output = self.inner_gru(input, hidden)
        output = output * mix + hidden * (1 - mix)

        return output

from typing import List, Tuple

class SkipGRULayer(jit.ScriptModule):
    __constants__ = ['hidden_size']
    def __init__(self, input_size, hidden_size):
        """
        Layer consisted of SkipGRU cells.
        Args:
            input_size (int): size of input space of GRU
            hidden_size (int): size of hidden space of GRU
        """
        super().__init__()
        self.cell = SkipGRUCell(input_size, hidden_size)
        self.hidden_size = hidden_size

    @jit.script_method
    def forward(self, input, mix):
        """
        Forward pass on SkipGRU layer.
        Args:
            input (torch.FloatTensor): inputs for the SkipGRU layer of form [BxTxN]
            mix (torch.FloatTensor): mixture coefficients for thehidden steps of form [BxTx1]
        Returns:
            h (torch.FloatTensor): history of outputs by steps. Size [BxTxH]
            o (torch.FloatTensor): last hidden step of size [BxH]
        """
        # type: (Tensor, Tensor) -> Tuple[Tensor, Tensor]
        hidden = torch.zeros(input.shape[0], self.hidden_size,
                             device=self.cell.inner_gru.weight_ih.device)
        inputs = input.unbind(1)
        mixtures = mix.unbind(1)
        outputs = jit.annotate(List[Tensor], [])
        for i in range(len(inputs)):
            hidden = self.cell(inputs[i], hidden, mixtures[i])
            outputs += [hidden]
        return torch.stack(outputs, 1), hidden


class SkipGRU(nn.Module):
    def __init__(self, input_size=None, hidden_size=None, layer=None, do_copy_weights=False):
        """
        SkipRNN layer with trivial binarizer (All betas are directly parametrized.)
        Args:
            input_size (int): size of input space for SkipGRU
            hidden_size (int): size of hidden space for SkipGRU
            layer (nn.GRU): original GRU layers to copy sizes and make a link to weights
        """
        super().__init__()
        if layer is not None:
            self.layer = SkipGRULayer(layer.input_size, layer.hidden_size)
            self.impute_weights(layer, do_copy_weights)
        elif (input_size is not None) and (hidden_size is not None):
            self.layer = SkipGRULayer(input_size, hidden_size)
        else:
            raise Exception('Either layer or input_size & hidden_size are required')

    def impute_weights(self, donor_layer, copy=False):
        """
        Get weights from instance of nn.GRU.
        Args:
            donor_layer (nn.GRU): pretrained layer to get weights from.
            copy (bool): if True, will make copy of weights instead of linkage.
        """
        assert isinstance(donor_layer, nn.GRU), 'Wrong type of donor layer. GRU required!'

        if copy:
            self.layer.cell.inner_gru.weight_ih.data = donor_layer.weight_ih_l0.clone()
            self.layer.cell.inner_gru.weight_hh.data = donor_layer.weight_hh_l0.clone()
            self.layer.cell.inner_gru.bias_ih.data = donor_layer.bias_ih_l0.clone()
            self.layer.cell.inner_gru.bias_hh.data = donor_layer.bias_hh_l0.clone()
        else:
            self.layer.cell.inner_gru.weight_ih = donor_layer.weight_ih_l0
            self.layer.cell.inner_gru.weight_hh = donor_layer.weight_hh_l0
            self.layer.cell.inner_gru.bias_ih = donor_layer.bias_ih_l0
            self.layer.cell.inner_gru.bias_hh = donor_layer.bias_hh_l0

    def forward(self, x, b, l=None):
        """
        Make forward SkipGRU pass.
        Args:
            x (torch.FloatTensor): input of size [BxTxN]. NB! sequences should be padded from the end, not from the start position.
            u (torch.FloatTensor): mixture coefficients of size [BxTx1]
            l (torch.LongTensor): lengths of the padded sequences. If not provided, output will contain last elements of sequences.
        Returns:
            h (torch.FloatTensor, nn.utils.rnn.PackedSequence): history of hidden states of size [BxTxH]
            o (torch.FloatTensor): last hidden state of size [BxH]
        """
        h, o = self.layer(x, b)
        if l is not None:
            o = h[torch.arange(h.shape[0]), l-1]
        return h, o

In [5]:
import torch
import numpy as np
from torch import nn
#from skiprnn import SkipGRU
import torch.functional as F

class IMDbRNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, num_classes):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = SkipGRU(embedding_size, hidden_size)
        self.cls = nn.Linear(hidden_size, num_classes)
        self.emb = nn.Embedding(vocab_size, embedding_size)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
        x, l = self.prepare_sequence(x)
        #на данном этапе учим с маской из одних единиц
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)
        return self.cls(o)

In [6]:
device = torch.device('cpu')
model = IMDbRNN(2000, 32, 16, 1)
model.to(device)

IMDbRNN(
  (rnn): SkipGRU(
    (layer): SkipGRULayer(
      original_name=SkipGRULayer
      (cell): SkipGRUCell(
        original_name=SkipGRUCell
        (inner_gru): ScriptModule(original_name=GRUCell)
      )
    )
  )
  (cls): Linear(in_features=16, out_features=1, bias=True)
  (emb): Embedding(2000, 32)
)

In [7]:
import torch
import torch.nn as nn
from catalyst.dl import SupervisedRunner
from catalyst.dl.callbacks import AUCCallback, F1ScoreCallback, CheckpointCallback, CriterionCallback, CriterionAggregatorCallback

/Users/ayunts/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [8]:
num_epochs = 100
logdir = "./logs/exp3"
loaders = {
    "train": train_loader,
    "valid": val_loader
}

criterion = {
   "bce": nn.BCEWithLogitsLoss()
}

optimizer = torch.optim.Adam([
    {'params': model.parameters(), 'lr': 3e-4}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner()

In [8]:
runner.train(
    model=model,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_bce",
                     criterion_key='bce', multiplier=1.),
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_bce']),
               AUCCallback(num_classes=1), 
                F1ScoreCallback(),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='auc/_mean',
    minimize_metric=False,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[2019-12-06 13:52:43,204] 
1/100 * Epoch 1 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=738.5268 | _timers/batch_time=0.7703 | _timers/data_time=0.0107 | _timers/model_time=0.7595 | auc/_mean=0.5220 | auc/class_0=0.5220 | f1_score=0.5011 | loss=0.6922 | loss_bce=0.6922
1/100 * Epoch 1 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=1178.0794 | _timers/batch_time=0.9482 | _timers/data_time=0.0192 | _timers/model_time=0.9290 | auc/_mean=0.5301 | auc/class_0=0.5301 | f1_score=0.3597 | loss=0.6911 | loss_bce=0.6911


KeyboardInterrupt: 

In [9]:
state = torch.load("best_skiprnn.pth",map_location='cpu')

model.load_state_dict(state['model_state_dict'])

<All keys matched successfully>

In [10]:
model.eval()

IMDbRNN(
  (rnn): SkipGRU(
    (layer): SkipGRULayer(
      original_name=SkipGRULayer
      (cell): SkipGRUCell(
        original_name=SkipGRUCell
        (inner_gru): ScriptModule(original_name=GRUCell)
      )
    )
  )
  (cls): Linear(in_features=16, out_features=1, bias=True)
  (emb): Embedding(2000, 32)
)

In [13]:
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score



In [14]:
predictions, labels = [], []
with torch.no_grad():
    for x,y in tqdm(test_loader):
        predictions.append(model(x).detach().cpu().numpy())
        labels.append(y.detach().cpu().numpy())
predictions = np.concatenate(predictions)
labels = np.concatenate(labels)

roc_auc_score(labels, predictions)

100%|██████████| 25/25 [00:21<00:00,  1.14it/s]


0.7049218304

## HintonBinarizer

In [15]:
from binarizers import HintonBinarizer

In [16]:
hb = HintonBinarizer()

In [17]:
for x,y in train_loader:
    break

In [18]:
x[0].shape

torch.Size([1306])

In [19]:
from torch.autograd import Variable


class IMDbRNN_bin(nn.Module):
    def __init__(self, sent_size):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = model.rnn
        self.cls = model.cls
        self.emb = model.emb
        
        self.u = Variable(torch.randn(1,sent_size,1), requires_grad=True)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
#         print(x.shape,x[0].shape)
        
        x = [i for i in x]
        x, l = self.prepare_sequence(x)
        
        #считаем с единичной маской
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)
        
        #считаем с бинаризованной маской u
        b_new = torch.repeat_interleave(hb(self.u), repeats=len(x), dim=0).to(self.emb.weight.device)
        h_new,o_new = self.rnn(x,b_new,l)
        
        #расстояние между выходами сети на полном и прореженом предложении
        dist = ((o-o_new)**2).sum(1)
        
        #среднее значение бинарной маски
        mean = b_new.squeeze(2).mean(1)
        
        #финальное предсказание класса на прореженом сообщении
        target = self.cls(o_new)
        
        return {'cls': target, 'dist': dist, 'mean': mean}

In [20]:
model_bin = IMDbRNN_bin(x[0].shape[0])

Делаем датасет из предложения x[0] размножив его несколько раз

In [21]:
from torch.utils.data import Dataset as BaseDataset

class Dataset(BaseDataset):

    
    def __init__(
            self, x, y, idx
    ):
        
        self.x = x
        self.y = y
        self.idx = idx
    
    def __getitem__(self, idx):

        return {'features': self.x[self.idx], 'target_cls':self.y[self.idx],
                'target_dist': torch.tensor(0).float(), 'target_mean': torch.tensor(0).float()}
        
    def __len__(self):
        return 8


new_train_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)
new_val_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)


In [22]:
num_epochs = 100  # change me
logdir = "./logs/Hinton"
loaders = {
    "train": new_train_loader,
    "valid": new_val_loader
}

criterion = {
   "mse": nn.MSELoss()
}

optimizer = torch.optim.Adam([
    {'params': model_bin.u, 'lr': 3e-3}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner(input_target_key=None,output_key=None)

In [23]:
runner.train(
    model=model_bin,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_dist", input_key='target_dist', output_key='dist',
                     criterion_key='mse', multiplier=0.5),
               CriterionCallback(prefix="loss_mean", input_key='target_mean', output_key='mean',
                     criterion_key='mse', multiplier=0.5),
        
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_dist','loss_mean']),
#                AUCCallback(num_classes=1, input_key='target_cls', output_key='cls'), 
                F1ScoreCallback(input_key='target_cls', output_key='cls'),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='loss',
    minimize_metric=True,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[2019-12-06 17:42:39,393] 
1/100 * Epoch 1 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=4.5196 | _timers/batch_time=0.4474 | _timers/data_time=0.0003 | _timers/model_time=0.4470 | f1_score=7.940e-07 | loss=0.2718 | loss_dist=0.2263 | loss_mean=0.0455
1/100 * Epoch 1 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=9.4056 | _timers/batch_time=0.2171 | _timers/data_time=0.0004 | _timers/model_time=0.2167 | f1_score=8.099e-07 | loss=0.2710 | loss_dist=0.2257 | loss_mean=0.0453
[2019-12-06 17:42:44,066] 
2/100 * Epoch 2 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=5.3819 | _timers/batch_time=0.3745 | _timers/data_time=0.0004 | _timers/model_time=0.3741 | f1_score=8.095e-07 | loss=0.2708 | loss_dist=0.2257 | loss_mean=0.0450
2/100 * Epoch 2 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=10.2587 | _timers/batch_time=0.1992 | _timers/data_time=0.0003 | _timers/model_time=0.1989 | f1_score=8.131e-07 | loss=0.2705 | loss_dist

[2019-12-06 17:43:54,046] 
17/100 * Epoch 17 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=5.6214 | _timers/batch_time=0.3564 | _timers/data_time=0.0003 | _timers/model_time=0.3560 | f1_score=7.030e-07 | loss=0.0643 | loss_dist=0.0283 | loss_mean=0.0361
17/100 * Epoch 17 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=10.4071 | _timers/batch_time=0.1962 | _timers/data_time=0.0002 | _timers/model_time=0.1960 | f1_score=7.785e-07 | loss=0.0597 | loss_dist=0.0238 | loss_mean=0.0359
[2019-12-06 17:43:58,409] 
18/100 * Epoch 18 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=5.7331 | _timers/batch_time=0.3503 | _timers/data_time=0.0004 | _timers/model_time=0.3498 | f1_score=7.897e-07 | loss=0.0592 | loss_dist=0.0235 | loss_mean=0.0357
18/100 * Epoch 18 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=10.3656 | _timers/batch_time=0.1969 | _timers/data_time=0.0004 | _timers/model_time=0.1965 | f1_score=8.861e-07 | loss=0.0579 | 

[2019-12-06 17:45:11,015] 
33/100 * Epoch 33 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=5.5506 | _timers/batch_time=0.3622 | _timers/data_time=0.0004 | _timers/model_time=0.3618 | f1_score=1.015e-06 | loss=0.0307 | loss_dist=0.0030 | loss_mean=0.0277
33/100 * Epoch 33 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=9.8237 | _timers/batch_time=0.2080 | _timers/data_time=0.0004 | _timers/model_time=0.2076 | f1_score=1.016e-06 | loss=0.0299 | loss_dist=0.0030 | loss_mean=0.0269
[2019-12-06 17:45:15,801] 
34/100 * Epoch 34 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=5.3750 | _timers/batch_time=0.3741 | _timers/data_time=0.0004 | _timers/model_time=0.3737 | f1_score=1.016e-06 | loss=0.0298 | loss_dist=0.0030 | loss_mean=0.0269
34/100 * Epoch 34 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=9.1978 | _timers/batch_time=0.2218 | _timers/data_time=0.0005 | _timers/model_time=0.2212 | f1_score=1.016e-06 | loss=0.0297 | lo

[2019-12-06 17:46:25,088] 
49/100 * Epoch 49 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=5.5588 | _timers/batch_time=0.3607 | _timers/data_time=0.0004 | _timers/model_time=0.3602 | f1_score=8.604e-07 | loss=0.0183 | loss_dist=0.0003 | loss_mean=0.0180
49/100 * Epoch 49 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=9.6411 | _timers/batch_time=0.2112 | _timers/data_time=0.0005 | _timers/model_time=0.2107 | f1_score=8.702e-07 | loss=0.0182 | loss_dist=0.0003 | loss_mean=0.0179
[2019-12-06 17:46:29,627] 
50/100 * Epoch 50 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=5.4898 | _timers/batch_time=0.3654 | _timers/data_time=0.0003 | _timers/model_time=0.3650 | f1_score=8.745e-07 | loss=0.0181 | loss_dist=0.0003 | loss_mean=0.0178
50/100 * Epoch 50 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=10.1425 | _timers/batch_time=0.2008 | _timers/data_time=0.0003 | _timers/model_time=0.2005 | f1_score=8.760e-07 | loss=0.0179 | l

[2019-12-06 17:47:40,243] 
65/100 * Epoch 65 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=5.1639 | _timers/batch_time=0.3884 | _timers/data_time=0.0005 | _timers/model_time=0.3879 | f1_score=1.036e-06 | loss=0.0121 | loss_dist=0.0005 | loss_mean=0.0116
65/100 * Epoch 65 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=9.8428 | _timers/batch_time=0.2077 | _timers/data_time=0.0004 | _timers/model_time=0.2073 | f1_score=1.031e-06 | loss=0.0118 | loss_dist=0.0005 | loss_mean=0.0113
[2019-12-06 17:47:44,964] 
66/100 * Epoch 66 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=5.4891 | _timers/batch_time=0.3665 | _timers/data_time=0.0003 | _timers/model_time=0.3661 | f1_score=1.030e-06 | loss=0.0117 | loss_dist=0.0005 | loss_mean=0.0112
66/100 * Epoch 66 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=9.8360 | _timers/batch_time=0.2084 | _timers/data_time=0.0004 | _timers/model_time=0.2080 | f1_score=1.030e-06 | loss=0.0117 | lo

[2019-12-06 17:48:57,603] 
81/100 * Epoch 81 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=5.5849 | _timers/batch_time=0.3593 | _timers/data_time=0.0004 | _timers/model_time=0.3588 | f1_score=1.089e-06 | loss=0.0091 | loss_dist=0.0011 | loss_mean=0.0080
81/100 * Epoch 81 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=10.2098 | _timers/batch_time=0.2010 | _timers/data_time=0.0003 | _timers/model_time=0.2007 | f1_score=1.020e-06 | loss=0.0087 | loss_dist=0.0006 | loss_mean=0.0081
[2019-12-06 17:49:02,288] 
82/100 * Epoch 82 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=5.4742 | _timers/batch_time=0.3679 | _timers/data_time=0.0004 | _timers/model_time=0.3675 | f1_score=9.896e-07 | loss=0.0086 | loss_dist=0.0005 | loss_mean=0.0080
82/100 * Epoch 82 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=9.7907 | _timers/batch_time=0.2093 | _timers/data_time=0.0003 | _timers/model_time=0.2089 | f1_score=9.588e-07 | loss=0.0083 | l

[2019-12-06 17:50:11,637] 
97/100 * Epoch 97 (train): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=5.6063 | _timers/batch_time=0.3583 | _timers/data_time=0.0004 | _timers/model_time=0.3578 | f1_score=1.168e-06 | loss=0.0092 | loss_dist=0.0027 | loss_mean=0.0065
97/100 * Epoch 97 (valid): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=10.2885 | _timers/batch_time=0.1979 | _timers/data_time=0.0003 | _timers/model_time=0.1976 | f1_score=1.168e-06 | loss=0.0091 | loss_dist=0.0027 | loss_mean=0.0064
[2019-12-06 17:50:16,135] 
98/100 * Epoch 98 (train): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=5.6234 | _timers/batch_time=0.3572 | _timers/data_time=0.0003 | _timers/model_time=0.3568 | f1_score=1.168e-06 | loss=0.0090 | loss_dist=0.0027 | loss_mean=0.0063
98/100 * Epoch 98 (valid): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=10.1119 | _timers/batch_time=0.2020 | _timers/data_time=0.0003 | _timers/model_time=0.2016 | f1_score=1.169e-06 | loss=0.0090 | 

In [48]:
model_bin.eval()
predictions, labels = [], []
with torch.no_grad():
    for x,y in tqdm(test_loader):
        try:
        
            predictions.append(model_bin(x)['cls'].detach().cpu().numpy())
            labels.append(y.detach().cpu().numpy())
        except: pass
predictions = np.concatenate(predictions)
labels = np.concatenate(labels)

roc_auc_score(labels, predictions)

100%|██████████| 25/25 [00:43<00:00,  1.75s/it]


0.5299492057635427

## ConcreteBinarizer

In [51]:
from binarizers import ConcreteBinarizer

In [52]:
cb = ConcreteBinarizer()

In [53]:
from torch.autograd import Variable


class IMDbRNN_bin(nn.Module):
    def __init__(self, sent_size):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = model.rnn
        self.cls = model.cls
        self.emb = model.emb
        
        self.u = Variable(torch.randn(1,sent_size,1), requires_grad=True)
        
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
#         print(x.shape,x[0].shape)
        
        x = [i for i in x]
        x, l = self.prepare_sequence(x)
        
        #считаем с единичной маской
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)
        
        #считаем с бинаризованной маской u
        b_new = torch.repeat_interleave(cb(self.u), repeats=len(x), dim=0).to(self.emb.weight.device)
        h_new,o_new = self.rnn(x,b_new,l)
        
        #расстояние между выходами сети на полном и прореженом предложении
        dist = ((o-o_new)**2).sum(1)
        
        #среднее значение бинарной маски
        mean = b_new.squeeze(2).mean(1)
        
        #финальное предсказание класса на прореженом сообщении
        target = self.cls(o_new)
        
        return {'cls': target, 'dist': dist, 'mean': mean}

In [54]:
model_bin = IMDbRNN_bin(x[0].shape[0])

In [55]:
from torch.utils.data import Dataset as BaseDataset

class Dataset(BaseDataset):

    
    def __init__(
            self, x, y, idx
    ):
        
        self.x = x
        self.y = y
        self.idx = idx
    
    def __getitem__(self, idx):

        return {'features': self.x[self.idx], 'target_cls':self.y[self.idx],
                'target_dist': torch.tensor(0).float(), 'target_mean': torch.tensor(0).float()}
        
    def __len__(self):
        return 8


new_train_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)
new_val_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)



In [56]:
num_epochs = 100  # change me
logdir = "./logs/Concrete"
loaders = {
    "train": new_train_loader,
    "valid": new_val_loader
}

criterion = {
   "mse": nn.MSELoss()
}

optimizer = torch.optim.Adam([
    {'params': model_bin.u, 'lr': 3e-3}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner(input_target_key=None,output_key=None)

In [57]:
runner.train(
    model=model_bin,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_dist", input_key='target_dist', output_key='dist',
                     criterion_key='mse', multiplier=0.5),
               CriterionCallback(prefix="loss_mean", input_key='target_mean', output_key='mean',
                     criterion_key='mse', multiplier=0.5),
        
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_dist','loss_mean']),
#                AUCCallback(num_classes=1, input_key='target_cls', output_key='cls'), 
                F1ScoreCallback(input_key='target_cls', output_key='cls'),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='loss',
    minimize_metric=True,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[2019-12-06 18:14:22,067] 
1/100 * Epoch 1 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=6.3213 | _timers/batch_time=0.3169 | _timers/data_time=0.0007 | _timers/model_time=0.3160 | f1_score=8.577e-07 | loss=3.2292 | loss_dist=3.1015 | loss_mean=0.1277
1/100 * Epoch 1 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=11.5877 | _timers/batch_time=0.1756 | _timers/data_time=0.0003 | _timers/model_time=0.1752 | f1_score=8.689e-07 | loss=3.1823 | loss_dist=3.0550 | loss_mean=0.1273
[2019-12-06 18:14:25,970] 
2/100 * Epoch 2 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=6.3997 | _timers/batch_time=0.3137 | _timers/data_time=0.0004 | _timers/model_time=0.3132 | f1_score=1.236e-06 | loss=0.6186 | loss_dist=0.4880 | loss_mean=0.1306
2/100 * Epoch 2 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=11.7097 | _timers/batch_time=0.1746 | _timers/data_time=0.0003 | _timers/model_time=0.1742 | f1_score=1.251e-06 | loss=0.6099 | loss_dis

[2019-12-06 18:15:27,177] 
17/100 * Epoch 17 (train): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=6.1392 | _timers/batch_time=0.3276 | _timers/data_time=0.0004 | _timers/model_time=0.3272 | f1_score=1.250e-06 | loss=0.3591 | loss_dist=0.2376 | loss_mean=0.1216
17/100 * Epoch 17 (valid): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=10.2200 | _timers/batch_time=0.2003 | _timers/data_time=0.0004 | _timers/model_time=0.1999 | f1_score=1.253e-06 | loss=0.3565 | loss_dist=0.2350 | loss_mean=0.1215
[2019-12-06 18:15:31,570] 
18/100 * Epoch 18 (train): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=5.7391 | _timers/batch_time=0.3519 | _timers/data_time=0.0003 | _timers/model_time=0.3515 | f1_score=1.081e-06 | loss=0.3549 | loss_dist=0.2339 | loss_mean=0.1209
18/100 * Epoch 18 (valid): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=10.6587 | _timers/batch_time=0.1932 | _timers/data_time=0.0004 | _timers/model_time=0.1928 | f1_score=1.082e-06 | loss=0.3529 | 

[2019-12-06 18:16:36,804] 
33/100 * Epoch 33 (train): _base/lr=0.0020 | _base/momentum=0.9000 | _timers/_fps=5.8860 | _timers/batch_time=0.3417 | _timers/data_time=0.0004 | _timers/model_time=0.3413 | f1_score=1.610e-06 | loss=0.2352 | loss_dist=0.1171 | loss_mean=0.1181
33/100 * Epoch 33 (valid): _base/lr=0.0020 | _base/momentum=0.9000 | _timers/_fps=10.5727 | _timers/batch_time=0.1930 | _timers/data_time=0.0002 | _timers/model_time=0.1928 | f1_score=1.617e-06 | loss=0.2343 | loss_dist=0.1164 | loss_mean=0.1179
[2019-12-06 18:16:40,962] 
34/100 * Epoch 34 (train): _base/lr=0.0020 | _base/momentum=0.9000 | _timers/_fps=6.0177 | _timers/batch_time=0.3330 | _timers/data_time=0.0003 | _timers/model_time=0.3327 | f1_score=1.056e-06 | loss=0.4652 | loss_dist=0.3510 | loss_mean=0.1141
34/100 * Epoch 34 (valid): _base/lr=0.0020 | _base/momentum=0.9000 | _timers/_fps=10.4361 | _timers/batch_time=0.1943 | _timers/data_time=0.0002 | _timers/model_time=0.1941 | f1_score=1.061e-06 | loss=0.4639 | 

[2019-12-06 18:17:40,722] 
49/100 * Epoch 49 (train): _base/lr=0.0016 | _base/momentum=0.9000 | _timers/_fps=6.5369 | _timers/batch_time=0.3073 | _timers/data_time=0.0004 | _timers/model_time=0.3069 | f1_score=1.292e-06 | loss=0.2648 | loss_dist=0.1536 | loss_mean=0.1111
49/100 * Epoch 49 (valid): _base/lr=0.0016 | _base/momentum=0.9000 | _timers/_fps=11.8972 | _timers/batch_time=0.1707 | _timers/data_time=0.0005 | _timers/model_time=0.1702 | f1_score=1.295e-06 | loss=0.2628 | loss_dist=0.1518 | loss_mean=0.1110
[2019-12-06 18:17:44,494] 
50/100 * Epoch 50 (train): _base/lr=0.0016 | _base/momentum=0.9000 | _timers/_fps=6.6405 | _timers/batch_time=0.3019 | _timers/data_time=0.0003 | _timers/model_time=0.3015 | f1_score=1.274e-06 | loss=0.2102 | loss_dist=0.0979 | loss_mean=0.1122
50/100 * Epoch 50 (valid): _base/lr=0.0016 | _base/momentum=0.9000 | _timers/_fps=11.8715 | _timers/batch_time=0.1709 | _timers/data_time=0.0004 | _timers/model_time=0.1705 | f1_score=1.278e-06 | loss=0.2096 | 

[2019-12-06 18:18:43,996] 
65/100 * Epoch 65 (train): _base/lr=0.0012 | _base/momentum=0.9000 | _timers/_fps=6.5917 | _timers/batch_time=0.3049 | _timers/data_time=0.0004 | _timers/model_time=0.3045 | f1_score=1.356e-06 | loss=0.4229 | loss_dist=0.3153 | loss_mean=0.1076
65/100 * Epoch 65 (valid): _base/lr=0.0012 | _base/momentum=0.9000 | _timers/_fps=11.7263 | _timers/batch_time=0.1728 | _timers/data_time=0.0003 | _timers/model_time=0.1725 | f1_score=1.357e-06 | loss=0.4227 | loss_dist=0.3152 | loss_mean=0.1075
[2019-12-06 18:18:47,871] 
66/100 * Epoch 66 (train): _base/lr=0.0012 | _base/momentum=0.9000 | _timers/_fps=6.5135 | _timers/batch_time=0.3088 | _timers/data_time=0.0003 | _timers/model_time=0.3085 | f1_score=1.132e-06 | loss=0.2220 | loss_dist=0.1130 | loss_mean=0.1089
66/100 * Epoch 66 (valid): _base/lr=0.0012 | _base/momentum=0.9000 | _timers/_fps=11.7317 | _timers/batch_time=0.1736 | _timers/data_time=0.0003 | _timers/model_time=0.1733 | f1_score=1.137e-06 | loss=0.2208 | 

[2019-12-06 18:19:47,320] 
81/100 * Epoch 81 (train): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=6.5210 | _timers/batch_time=0.3070 | _timers/data_time=0.0004 | _timers/model_time=0.3066 | f1_score=1.667e-06 | loss=0.3028 | loss_dist=0.1985 | loss_mean=0.1042
81/100 * Epoch 81 (valid): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=11.8073 | _timers/batch_time=0.1724 | _timers/data_time=0.0002 | _timers/model_time=0.1722 | f1_score=1.668e-06 | loss=0.3023 | loss_dist=0.1982 | loss_mean=0.1041
[2019-12-06 18:19:51,106] 
82/100 * Epoch 82 (train): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=6.6520 | _timers/batch_time=0.3021 | _timers/data_time=0.0004 | _timers/model_time=0.3016 | f1_score=1.797e-06 | loss=0.1549 | loss_dist=0.0452 | loss_mean=0.1097
82/100 * Epoch 82 (valid): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=11.9542 | _timers/batch_time=0.1707 | _timers/data_time=0.0003 | _timers/model_time=0.1704 | f1_score=1.798e-06 | loss=0.1547 | 

[2019-12-06 18:20:50,280] 
97/100 * Epoch 97 (train): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=6.2791 | _timers/batch_time=0.3233 | _timers/data_time=0.0004 | _timers/model_time=0.3229 | f1_score=1.407e-06 | loss=0.1394 | loss_dist=0.0387 | loss_mean=0.1007
97/100 * Epoch 97 (valid): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=11.1642 | _timers/batch_time=0.1810 | _timers/data_time=0.0003 | _timers/model_time=0.1806 | f1_score=1.408e-06 | loss=0.1391 | loss_dist=0.0384 | loss_mean=0.1007
[2019-12-06 18:20:54,417] 
98/100 * Epoch 98 (train): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=6.2374 | _timers/batch_time=0.3245 | _timers/data_time=0.0003 | _timers/model_time=0.3241 | f1_score=1.528e-06 | loss=0.1387 | loss_dist=0.0387 | loss_mean=0.1000
98/100 * Epoch 98 (valid): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=10.8333 | _timers/batch_time=0.1879 | _timers/data_time=0.0003 | _timers/model_time=0.1876 | f1_score=1.529e-06 | loss=0.1391 | 

In [58]:
model_bin.eval()
predictions, labels = [], []
with torch.no_grad():
    for x,y in tqdm(test_loader):
        try:
        
            predictions.append(model_bin(x)['cls'].detach().cpu().numpy())
            labels.append(y.detach().cpu().numpy())
        except: pass
predictions = np.concatenate(predictions)
labels = np.concatenate(labels)

roc_auc_score(labels, predictions)

100%|██████████| 25/25 [00:36<00:00,  1.45s/it]


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

### HardKumaBinarizer

In [59]:
from binarizers import HardKumaBinarizer

In [60]:
hkb = HardKumaBinarizer()

In [61]:
from torch.autograd import Variable


class IMDbRNN_bin(nn.Module):
    def __init__(self, sent_size):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = model.rnn
        self.cls = model.cls
        self.emb = model.emb
        
        self.u = Variable(torch.randn(1,sent_size,1), requires_grad=True)
        self.v = Variable(torch.randn(1,sent_size,1), requires_grad=True)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
#         print(x.shape,x[0].shape)
        
        x = [i for i in x]
        x, l = self.prepare_sequence(x)
        
        #считаем с единичной маской
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)
        
        #считаем с бинаризованной маской u
        b_new = torch.repeat_interleave(hkb(self.u, self.v), repeats=len(x), dim=0).to(self.emb.weight.device)
        h_new,o_new = self.rnn(x,b_new,l)
        
        #расстояние между выходами сети на полном и прореженом предложении
        dist = ((o-o_new)**2).sum(1)
        
        #среднее значение бинарной маски
        mean = b_new.squeeze(2).mean(1)
        
        #финальное предсказание класса на прореженом сообщении
        target = self.cls(o_new)
        
        return {'cls': target, 'dist': dist, 'mean': mean}

In [62]:
model_bin = IMDbRNN_bin(x[0].shape[0])
class Dataset(BaseDataset):

    
    def __init__(
            self, x, y, idx
    ):
        
        self.x = x
        self.y = y
        self.idx = idx
    
    def __getitem__(self, idx):

        return {'features': self.x[self.idx], 'target_cls':self.y[self.idx],
                'target_dist': torch.tensor(0).float(), 'target_mean': torch.tensor(0).float()}
        
    def __len__(self):
        return 8


new_train_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)
new_val_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)

In [63]:
num_epochs = 100  # change me
logdir = "./logs/HardKuma"
loaders = {
    "train": new_train_loader,
    "valid": new_val_loader
}

criterion = {
   "mse": nn.MSELoss()
}

optimizer = torch.optim.Adam([
    {'params': model_bin.u, 'lr': 3e-3}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner(input_target_key=None,output_key=None)

In [64]:
runner.train(
    model=model_bin,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_dist", input_key='target_dist', output_key='dist',
                     criterion_key='mse', multiplier=0.5),
               CriterionCallback(prefix="loss_mean", input_key='target_mean', output_key='mean',
                     criterion_key='mse', multiplier=0.5),
        
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_dist','loss_mean']),
#                AUCCallback(num_classes=1, input_key='target_cls', output_key='cls'), 
                F1ScoreCallback(input_key='target_cls', output_key='cls'),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='loss',
    minimize_metric=True,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[2019-12-06 18:22:44,807] 
1/100 * Epoch 1 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=5.8000 | _timers/batch_time=0.3458 | _timers/data_time=0.0002 | _timers/model_time=0.3455 | f1_score=5.300e-07 | loss=2.6945 | loss_dist=2.5584 | loss_mean=0.1361
1/100 * Epoch 1 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=10.9563 | _timers/batch_time=0.1864 | _timers/data_time=0.0002 | _timers/model_time=0.1861 | f1_score=5.511e-07 | loss=2.5137 | loss_dist=2.3779 | loss_mean=0.1358
[2019-12-06 18:22:49,198] 
2/100 * Epoch 2 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=5.8326 | _timers/batch_time=0.3468 | _timers/data_time=0.0003 | _timers/model_time=0.3465 | f1_score=6.214e-07 | loss=13.2910 | loss_dist=13.1538 | loss_mean=0.1372
2/100 * Epoch 2 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=11.0507 | _timers/batch_time=0.1853 | _timers/data_time=0.0002 | _timers/model_time=0.1850 | f1_score=6.311e-07 | loss=13.0239 | loss_

[2019-12-06 18:23:52,440] 
17/100 * Epoch 17 (train): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=6.3261 | _timers/batch_time=0.3163 | _timers/data_time=0.0003 | _timers/model_time=0.3160 | f1_score=1.320e-06 | loss=0.5241 | loss_dist=0.3872 | loss_mean=0.1368
17/100 * Epoch 17 (valid): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=11.5995 | _timers/batch_time=0.1752 | _timers/data_time=0.0004 | _timers/model_time=0.1747 | f1_score=1.328e-06 | loss=0.5137 | loss_dist=0.3769 | loss_mean=0.1368
[2019-12-06 18:23:56,342] 
18/100 * Epoch 18 (train): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=6.5595 | _timers/batch_time=0.3056 | _timers/data_time=0.0002 | _timers/model_time=0.3053 | f1_score=1.206e-06 | loss=0.3433 | loss_dist=0.2080 | loss_mean=0.1353
18/100 * Epoch 18 (valid): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=11.2003 | _timers/batch_time=0.1855 | _timers/data_time=0.0005 | _timers/model_time=0.1850 | f1_score=1.218e-06 | loss=0.3396 | 

[2019-12-06 18:24:58,511] 
33/100 * Epoch 33 (train): _base/lr=0.0020 | _base/momentum=0.9000 | _timers/_fps=6.2059 | _timers/batch_time=0.3257 | _timers/data_time=0.0003 | _timers/model_time=0.3254 | f1_score=1.351e-06 | loss=0.2424 | loss_dist=0.1057 | loss_mean=0.1368
33/100 * Epoch 33 (valid): _base/lr=0.0020 | _base/momentum=0.9000 | _timers/_fps=10.7376 | _timers/batch_time=0.1920 | _timers/data_time=0.0004 | _timers/model_time=0.1916 | f1_score=1.355e-06 | loss=0.2412 | loss_dist=0.1045 | loss_mean=0.1368
[2019-12-06 18:25:02,837] 
34/100 * Epoch 34 (train): _base/lr=0.0020 | _base/momentum=0.9000 | _timers/_fps=5.8015 | _timers/batch_time=0.3492 | _timers/data_time=0.0004 | _timers/model_time=0.3486 | f1_score=1.125e-06 | loss=0.3071 | loss_dist=0.1724 | loss_mean=0.1347
34/100 * Epoch 34 (valid): _base/lr=0.0020 | _base/momentum=0.9000 | _timers/_fps=11.6071 | _timers/batch_time=0.1759 | _timers/data_time=0.0003 | _timers/model_time=0.1756 | f1_score=1.130e-06 | loss=0.3044 | 

[2019-12-06 18:26:06,689] 
49/100 * Epoch 49 (train): _base/lr=0.0016 | _base/momentum=0.9000 | _timers/_fps=5.6938 | _timers/batch_time=0.3531 | _timers/data_time=0.0004 | _timers/model_time=0.3526 | f1_score=1.558e-06 | loss=0.2376 | loss_dist=0.1026 | loss_mean=0.1350
49/100 * Epoch 49 (valid): _base/lr=0.0016 | _base/momentum=0.9000 | _timers/_fps=10.0012 | _timers/batch_time=0.2033 | _timers/data_time=0.0003 | _timers/model_time=0.2030 | f1_score=1.560e-06 | loss=0.2367 | loss_dist=0.1017 | loss_mean=0.1349
[2019-12-06 18:26:11,242] 
50/100 * Epoch 50 (train): _base/lr=0.0016 | _base/momentum=0.9000 | _timers/_fps=5.6107 | _timers/batch_time=0.3578 | _timers/data_time=0.0004 | _timers/model_time=0.3573 | f1_score=1.421e-06 | loss=0.2255 | loss_dist=0.0921 | loss_mean=0.1334
50/100 * Epoch 50 (valid): _base/lr=0.0016 | _base/momentum=0.9000 | _timers/_fps=10.4859 | _timers/batch_time=0.1940 | _timers/data_time=0.0002 | _timers/model_time=0.1938 | f1_score=1.424e-06 | loss=0.2244 | 

[2019-12-06 18:27:11,801] 
65/100 * Epoch 65 (train): _base/lr=0.0013 | _base/momentum=0.9000 | _timers/_fps=5.8216 | _timers/batch_time=0.3437 | _timers/data_time=0.0002 | _timers/model_time=0.3435 | f1_score=1.479e-06 | loss=0.3088 | loss_dist=0.1752 | loss_mean=0.1335
65/100 * Epoch 65 (valid): _base/lr=0.0013 | _base/momentum=0.9000 | _timers/_fps=7.4523 | _timers/batch_time=0.2735 | _timers/data_time=0.0005 | _timers/model_time=0.2729 | f1_score=1.481e-06 | loss=0.3077 | loss_dist=0.1742 | loss_mean=0.1335
[2019-12-06 18:27:16,659] 
66/100 * Epoch 66 (train): _base/lr=0.0013 | _base/momentum=0.9000 | _timers/_fps=5.1382 | _timers/batch_time=0.3932 | _timers/data_time=0.0004 | _timers/model_time=0.3928 | f1_score=1.421e-06 | loss=0.2137 | loss_dist=0.0807 | loss_mean=0.1331
66/100 * Epoch 66 (valid): _base/lr=0.0013 | _base/momentum=0.9000 | _timers/_fps=11.2467 | _timers/batch_time=0.1813 | _timers/data_time=0.0003 | _timers/model_time=0.1810 | f1_score=1.423e-06 | loss=0.2127 | l

[2019-12-06 18:28:18,288] 
81/100 * Epoch 81 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=6.4692 | _timers/batch_time=0.3110 | _timers/data_time=0.0003 | _timers/model_time=0.3106 | f1_score=1.437e-06 | loss=0.3270 | loss_dist=0.1945 | loss_mean=0.1325
81/100 * Epoch 81 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=11.7204 | _timers/batch_time=0.1740 | _timers/data_time=0.0004 | _timers/model_time=0.1736 | f1_score=1.439e-06 | loss=0.3255 | loss_dist=0.1931 | loss_mean=0.1324
[2019-12-06 18:28:22,072] 
82/100 * Epoch 82 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=6.6110 | _timers/batch_time=0.3032 | _timers/data_time=0.0003 | _timers/model_time=0.3029 | f1_score=1.640e-06 | loss=0.1962 | loss_dist=0.0603 | loss_mean=0.1358
82/100 * Epoch 82 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=11.8844 | _timers/batch_time=0.1712 | _timers/data_time=0.0003 | _timers/model_time=0.1708 | f1_score=1.642e-06 | loss=0.1958 | 

[2019-12-06 18:29:26,015] 
97/100 * Epoch 97 (train): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=5.8393 | _timers/batch_time=0.3446 | _timers/data_time=0.0003 | _timers/model_time=0.3443 | f1_score=1.518e-06 | loss=0.1814 | loss_dist=0.0481 | loss_mean=0.1333
97/100 * Epoch 97 (valid): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=10.6915 | _timers/batch_time=0.1938 | _timers/data_time=0.0002 | _timers/model_time=0.1935 | f1_score=1.520e-06 | loss=0.1810 | loss_dist=0.0477 | loss_mean=0.1333
[2019-12-06 18:29:30,058] 
98/100 * Epoch 98 (train): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=6.2063 | _timers/batch_time=0.3240 | _timers/data_time=0.0003 | _timers/model_time=0.3236 | f1_score=1.525e-06 | loss=0.2151 | loss_dist=0.0839 | loss_mean=0.1312
98/100 * Epoch 98 (valid): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=11.0639 | _timers/batch_time=0.1841 | _timers/data_time=0.0004 | _timers/model_time=0.1836 | f1_score=1.526e-06 | loss=0.2146 | 

### Bernoulli p = 0.5

In [66]:
from torch.autograd import Variable


class IMDbRNN_bin(nn.Module):
    def __init__(self, sent_size):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = model.rnn
        self.cls = model.cls
        self.emb = model.emb
        
        self.u = Variable(torch.randn(1,sent_size,1), requires_grad=True)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
#         print(x.shape,x[0].shape)
        
        x = [i for i in x]
        x, l = self.prepare_sequence(x)
        
        #считаем с единичной маской
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)               
        
        #считаем с бинаризованной маской u
        #torch.bernoulli(torch.tensor([1.0]),)
        b_new = torch.bernoulli(torch.ones(x.shape[:2]),0.5).unsqueeze(2).to(self.emb.weight.device)
        #b_new = torch.repeat_interleave(bb(0.4), repeats=len(x), dim=0).to(self.emb.weight.device)
        h_new,o_new = self.rnn(x,b_new,l)
        
        #расстояние между выходами сети на полном и прореженом предложении
        dist = ((o-o_new)**2).sum(1)
        
        #среднее значение бинарной маски
        mean = b_new.squeeze(2).mean(1)
        
        #финальное предсказание класса на прореженом сообщении
        target = self.cls(o_new)
        
        return {'cls': target, 'dist': dist, 'mean': mean}

In [67]:
model_bin = IMDbRNN_bin(x[0].shape[0])
class Dataset(BaseDataset):

    
    def __init__(
            self, x, y, idx
    ):
        
        self.x = x
        self.y = y
        self.idx = idx
    
    def __getitem__(self, idx):

        return {'features': self.x[self.idx], 'target_cls':self.y[self.idx],
                'target_dist': torch.tensor(0).float(), 'target_mean': torch.tensor(0).float()}
        
    def __len__(self):
        return 8


new_train_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)
new_val_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)

In [68]:
num_epochs = 100  # change me
logdir = "./logs/b50"
loaders = {
    "train": new_train_loader,
    "valid": new_val_loader
}

criterion = {
   "mse": nn.MSELoss()
}

optimizer = torch.optim.Adam([
    {'params': model_bin.u, 'lr': 3e-3}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner(input_target_key=None,output_key=None)

In [ ]:
runner.train(
    model=model_bin,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_dist", input_key='target_dist', output_key='dist',
                     criterion_key='mse', multiplier=0.5),
               CriterionCallback(prefix="loss_mean", input_key='target_mean', output_key='mean',
                     criterion_key='mse', multiplier=0.5),
        
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_dist','loss_mean']),
#                AUCCallback(num_classes=1, input_key='target_cls', output_key='cls'), 
                F1ScoreCallback(input_key='target_cls', output_key='cls'),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='loss',
    minimize_metric=True,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model_bin.eval()
predictions, labels = [], []
with torch.no_grad():
    for x,y in tqdm(test_loader):
        try:
        
            predictions.append(model_bin(x)['cls'].detach().cpu().numpy())
            labels.append(y.detach().cpu().numpy())
        except: pass
predictions = np.concatenate(predictions)
labels = np.concatenate(labels)

roc_auc_score(labels, predictions)

### Bernoulli p = 0.9

In [ ]:
p = 0.9

In [ ]:
from torch.autograd import Variable


class IMDbRNN_bin(nn.Module):
    def __init__(self, sent_size):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = model.rnn
        self.cls = model.cls
        self.emb = model.emb
        
        self.u = Variable(torch.randn(1,sent_size,1), requires_grad=True)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
#         print(x.shape,x[0].shape)
        
        x = [i for i in x]
        x, l = self.prepare_sequence(x)
        
        #считаем с единичной маской
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)               
        
        #считаем с бинаризованной маской u
        #torch.bernoulli(torch.tensor([1.0]),)
        b_new = torch.bernoulli(torch.ones(x.shape[:2]),p).unsqueeze(2).to(self.emb.weight.device)
        #b_new = torch.repeat_interleave(bb(0.4), repeats=len(x), dim=0).to(self.emb.weight.device)
        h_new,o_new = self.rnn(x,b_new,l)
        
        #расстояние между выходами сети на полном и прореженом предложении
        dist = ((o-o_new)**2).sum(1)
        
        #среднее значение бинарной маски
        mean = b_new.squeeze(2).mean(1)
        
        #финальное предсказание класса на прореженом сообщении
        target = self.cls(o_new)
        
        return {'cls': target, 'dist': dist, 'mean': mean}

In [ ]:
model_bin = IMDbRNN_bin(x[0].shape[0])
class Dataset(BaseDataset):

    
    def __init__(
            self, x, y, idx
    ):
        
        self.x = x
        self.y = y
        self.idx = idx
    
    def __getitem__(self, idx):

        return {'features': self.x[self.idx], 'target_cls':self.y[self.idx],
                'target_dist': torch.tensor(0).float(), 'target_mean': torch.tensor(0).float()}
        
    def __len__(self):
        return 8


new_train_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)
new_val_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)

In [ ]:
num_epochs = 100  # change me
logdir = "./logs/b90"
loaders = {
    "train": new_train_loader,
    "valid": new_val_loader
}

criterion = {
   "mse": nn.MSELoss()
}

optimizer = torch.optim.Adam([
    {'params': model_bin.u, 'lr': 3e-3}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner(input_target_key=None,output_key=None)

In [ ]:
runner.train(
    model=model_bin,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_dist", input_key='target_dist', output_key='dist',
                     criterion_key='mse', multiplier=0.5),
               CriterionCallback(prefix="loss_mean", input_key='target_mean', output_key='mean',
                     criterion_key='mse', multiplier=0.5),
        
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_dist','loss_mean']),
#                AUCCallback(num_classes=1, input_key='target_cls', output_key='cls'), 
                F1ScoreCallback(input_key='target_cls', output_key='cls'),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='loss',
    minimize_metric=True,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

In [ ]:
model_bin.eval()
predictions, labels = [], []
with torch.no_grad():
    for x,y in tqdm(test_loader):
        try:
        
            predictions.append(model_bin(x)['cls'].detach().cpu().numpy())
            labels.append(y.detach().cpu().numpy())
        except: pass
predictions = np.concatenate(predictions)
labels = np.concatenate(labels)

roc_auc_score(labels, predictions)

### Bernoulli p = 0.2

In [ ]:
p = 0.2

In [ ]:
from torch.autograd import Variable


class IMDbRNN_bin(nn.Module):
    def __init__(self, sent_size):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = model.rnn
        self.cls = model.cls
        self.emb = model.emb
        
        self.u = Variable(torch.randn(1,sent_size,1), requires_grad=True)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
#         print(x.shape,x[0].shape)
        
        x = [i for i in x]
        x, l = self.prepare_sequence(x)
        
        #считаем с единичной маской
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)               
        
        #считаем с бинаризованной маской u
        #torch.bernoulli(torch.tensor([1.0]),)
        b_new = torch.bernoulli(torch.ones(x.shape[:2]),p).unsqueeze(2).to(self.emb.weight.device)
        #b_new = torch.repeat_interleave(bb(0.4), repeats=len(x), dim=0).to(self.emb.weight.device)
        h_new,o_new = self.rnn(x,b_new,l)
        
        #расстояние между выходами сети на полном и прореженом предложении
        dist = ((o-o_new)**2).sum(1)
        
        #среднее значение бинарной маски
        mean = b_new.squeeze(2).mean(1)
        
        #финальное предсказание класса на прореженом сообщении
        target = self.cls(o_new)
        
        return {'cls': target, 'dist': dist, 'mean': mean}

In [ ]:
model_bin = IMDbRNN_bin(x[0].shape[0])
class Dataset(BaseDataset):

    
    def __init__(
            self, x, y, idx
    ):
        
        self.x = x
        self.y = y
        self.idx = idx
    
    def __getitem__(self, idx):

        return {'features': self.x[self.idx], 'target_cls':self.y[self.idx],
                'target_dist': torch.tensor(0).float(), 'target_mean': torch.tensor(0).float()}
        
    def __len__(self):
        return 8


new_train_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)
new_val_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)

In [ ]:
num_epochs = 100  # change me
logdir = "./logs/b20"
loaders = {
    "train": new_train_loader,
    "valid": new_val_loader
}

criterion = {
   "mse": nn.MSELoss()
}

optimizer = torch.optim.Adam([
    {'params': model_bin.u, 'lr': 3e-3}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner(input_target_key=None,output_key=None)

In [ ]:
runner.train(
    model=model_bin,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_dist", input_key='target_dist', output_key='dist',
                     criterion_key='mse', multiplier=0.5),
               CriterionCallback(prefix="loss_mean", input_key='target_mean', output_key='mean',
                     criterion_key='mse', multiplier=0.5),
        
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_dist','loss_mean']),
#                AUCCallback(num_classes=1, input_key='target_cls', output_key='cls'), 
                F1ScoreCallback(input_key='target_cls', output_key='cls'),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='loss',
    minimize_metric=True,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

In [ ]:
model_bin.eval()
predictions, labels = [], []
with torch.no_grad():
    for x,y in tqdm(test_loader):
        try:
        
            predictions.append(model_bin(x)['cls'].detach().cpu().numpy())
            labels.append(y.detach().cpu().numpy())
        except: pass
predictions = np.concatenate(predictions)
labels = np.concatenate(labels)

roc_auc_score(labels, predictions)

### Bernoulli p = 0.8

In [ ]:
p = 0.8

In [ ]:
from torch.autograd import Variable


class IMDbRNN_bin(nn.Module):
    def __init__(self, sent_size):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = model.rnn
        self.cls = model.cls
        self.emb = model.emb
        
        self.u = Variable(torch.randn(1,sent_size,1), requires_grad=True)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
#         print(x.shape,x[0].shape)
        
        x = [i for i in x]
        x, l = self.prepare_sequence(x)
        
        #считаем с единичной маской
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)               
        
        #считаем с бинаризованной маской u
        #torch.bernoulli(torch.tensor([1.0]),)
        b_new = torch.bernoulli(torch.ones(x.shape[:2]),p).unsqueeze(2).to(self.emb.weight.device)
        #b_new = torch.repeat_interleave(bb(0.4), repeats=len(x), dim=0).to(self.emb.weight.device)
        h_new,o_new = self.rnn(x,b_new,l)
        
        #расстояние между выходами сети на полном и прореженом предложении
        dist = ((o-o_new)**2).sum(1)
        
        #среднее значение бинарной маски
        mean = b_new.squeeze(2).mean(1)
        
        #финальное предсказание класса на прореженом сообщении
        target = self.cls(o_new)
        
        return {'cls': target, 'dist': dist, 'mean': mean}

In [ ]:
model_bin = IMDbRNN_bin(x[0].shape[0])
class Dataset(BaseDataset):

    
    def __init__(
            self, x, y, idx
    ):
        
        self.x = x
        self.y = y
        self.idx = idx
    
    def __getitem__(self, idx):

        return {'features': self.x[self.idx], 'target_cls':self.y[self.idx],
                'target_dist': torch.tensor(0).float(), 'target_mean': torch.tensor(0).float()}
        
    def __len__(self):
        return 8


new_train_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)
new_val_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)

In [ ]:
num_epochs = 100  # change me
logdir = "./logs/b80"
loaders = {
    "train": new_train_loader,
    "valid": new_val_loader
}

criterion = {
   "mse": nn.MSELoss()
}

optimizer = torch.optim.Adam([
    {'params': model_bin.u, 'lr': 3e-3}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner(input_target_key=None,output_key=None)

In [ ]:
runner.train(
    model=model_bin,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_dist", input_key='target_dist', output_key='dist',
                     criterion_key='mse', multiplier=0.5),
               CriterionCallback(prefix="loss_mean", input_key='target_mean', output_key='mean',
                     criterion_key='mse', multiplier=0.5),
        
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_dist','loss_mean']),
#                AUCCallback(num_classes=1, input_key='target_cls', output_key='cls'), 
                F1ScoreCallback(input_key='target_cls', output_key='cls'),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='loss',
    minimize_metric=True,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

In [ ]:
model_bin.eval()
predictions, labels = [], []
with torch.no_grad():
    for x,y in tqdm(test_loader):
        try:
        
            predictions.append(model_bin(x)['cls'].detach().cpu().numpy())
            labels.append(y.detach().cpu().numpy())
        except: pass
predictions = np.concatenate(predictions)
labels = np.concatenate(labels)

roc_auc_score(labels, predictions)